In [ ]:
#Training pipeline for resnet-50 classificaiton model by Tomas Slaven of University Of Cape Town
#ResNet architecture derived from https://pytorch.org/hub/nvidia_deeplearningexamples_resnet50

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import ast
import argparse
import os

#Import Google Drive folder, assuming the use of google colab
from google.colab import drive
drive.mount('/content/drive')

# Declare highest_mean_accuracy and highest macro F1-Score as global variables
global highest_mean_accuracy
global highest_f1_score
highest_mean_accuracy = 0.0
highest_f1_score = 0.0


# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, image_dir, csv_file, transform=None):
        self.image_dir = image_dir
        self.data = pd.read_csv(csv_file, encoding="utf-8")
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_id = self.data.iloc[idx, 1]
        image_path = f"{self.image_dir}/{image_id}.jpeg"
        image = Image.open(image_path).convert("RGB")
        label = torch.tensor(self.data.iloc[idx, 2:], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

#Function to load resnet model
def load_resnet50(num_classes):
    #model = models.resnet50(pretrained=True)
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

# Function to load data
def load_data(image_dir, train_csv, validation_csv, test_csv, batch_size, resolution):
    transform = transforms.Compose(
        [
            transforms.Resize((resolution, resolution)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.3721, 0.3721, 0.3721], std=[0.1801, 0.1801, 0.1801]),
        ]
    )

    train_dataset = CustomDataset(image_dir, train_csv, transform)
    validation_dataset = CustomDataset(image_dir, validation_csv, transform)
    test_dataset = CustomDataset(image_dir, test_csv, transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

    return train_loader, validation_loader, test_loader


# Function to train the model
def train_model(
    model, criterion, optimizer, train_loader, validation_loader, num_epochs,
    device, output_folder_path
):
    global highest_mean_accuracy
    global highest_f1_score
    highest_mean_accuracy = 0.0
    highest_f1_score = 0.0
    model.to(device)
    all_epoch_metrics = []
    val_epoch_metrics = []
    train_df = pd.DataFrame()
    val_df = pd.DataFrame()

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        total_correct = 0
        label_correct = [0] * args.num_classes
        label_total = [0] * args.num_classes
        TP = [0] * args.num_classes
        FP = [0] * args.num_classes
        TN = [0] * args.num_classes
        FN = [0] * args.num_classes
        precision, specificity, sensi, FNR, F1, label_accuracy =  [], [], [], [], [], []
        print("Epoch: ", epoch)
        print("")

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

            # Calculate accuracies
            predicted_labels = (torch.sigmoid(outputs) > 0.5).float()
            total_correct += (predicted_labels == labels).all(dim=1).sum().item()
            correct_per_label = (predicted_labels == labels).sum(dim=0).tolist()

            for i in range(args.num_classes):
                label_correct[i] += correct_per_label[i]
                label_total[i] += len(images)

                #calc TP and FP
                true_positive = ((predicted_labels[:, i] == 1) & (labels[:, i] == 1)).sum().item()
                false_positive = ((predicted_labels[:, i] == 1) & (labels[:, i] == 0)).sum().item()
                true_negative = ((predicted_labels[:, i] == 0) & (labels[:, i] == 0)).sum().item()
                false_negative = ((predicted_labels[:, i] == 0) & (labels[:, i] == 1)).sum().item()

                TP[i] += true_positive
                FP[i] += false_positive
                TN[i] += true_negative
                FN[i] += false_negative

        #Calculate Label Specific Metrics
        for i in range(args.num_classes):
            label_accuracy.append(
                (label_correct[i] / label_total[i] if label_total[i] > 0 else 0)*100
            )
            precision.append((TP[i] / (TP[i] + FP[i]) if TP[i] + FP[i] > 0 else 0)*100)
            sensi.append((TP[i] / (TP[i] + FN[i]) if TP[i] + FN[i] > 0 else 0)*100)
            specificity.append((TN[i] / (TN[i] + FP[i]) if TN[i] + FP[i] > 0 else 0)*100)
            FNR.append((FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0)*100)
            F1.append( 2 * (precision[i] * sensi[i]) / (precision[i] + sensi[i]) if precision[i] + sensi[i] > 0 else 0)

        # Calculate macro-averages and overall training accuracy
        train_loss /= len(train_loader.dataset)
        overall_accuracy = total_correct / len(train_loader.dataset)
        train_accuracy = overall_accuracy * 100

        label_accuracy.insert(0, train_accuracy )
        precision.insert(0, (sum(precision) / args.num_classes))
        sensi.insert(0, (sum(sensi) / args.num_classes))
        specificity.insert(0, (sum(specificity) / args.num_classes))
        FNR.insert(0, (sum(FNR) / args.num_classes))
        F1_score = (sum(F1) / args.num_classes)
        F1.insert(0, (sum(F1) / args.num_classes))

        # Create a dictionary for training metrics and save to DataFrame
        train_metrics_dict = {
            "Epoch": [epoch] * len(label_names),
            "Type (TRAIN)": label_names,
            "Mean Accuracy (TRAIN)": label_accuracy,
            "Precision (TRAIN)": precision,
            "Sensitivity (TRAIN)": sensi,
            "Specificity (TRAIN)": specificity,
            "FNR (TRAIN)": FNR,
            "F1 Score (TRAIN)": F1,
        }
        a = pd.DataFrame(train_metrics_dict)
        train_df = pd.concat([train_df, a], axis=0)

        print("Training Label Accuracies:")
        print(f"Overall Accuracy: {train_accuracy:.4f}")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"F1 Score: {F1_score:.4f}")
        print("")

        # Validate the model and get validation metrics
        val_metrics_dict = validate_model(
            model, criterion, validation_loader, DEVICE, output_folder_path, epoch
        )

        b = pd.DataFrame(val_metrics_dict)
        val_df = pd.concat([val_df, b], axis=0)


    # Concatenate all metric DataFrames vertically
    combined_df = pd.concat([train_df, val_df ], axis=1)

    return combined_df


# Function to validate the model
def validate_model(model, criterion, validation_loader, device, output_folder_path, epoch):
    model.eval()
    global highest_mean_accuracy
    global highest_f1_score


    with torch.no_grad():
        validation_loss = 0.0
        total_correct = 0
        label_correct = [0] * args.num_classes
        label_total = [0] * args.num_classes
        TP = [0] * args.num_classes
        FP = [0] * args.num_classes
        TN = [0] * args.num_classes
        FN = [0] * args.num_classes
        precision, specificity, sensi, FNR, F1, label_accuracy =  [], [], [], [], [], []

        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item() * images.size(0)


            # Calculate accuracies
            predicted_labels = (torch.sigmoid(outputs) > 0.5).float()
            total_correct += (predicted_labels == labels).all(dim=1).sum().item()
            correct_per_label = (predicted_labels == labels).sum(dim=0).tolist()

            for i in range(args.num_classes):
                label_correct[i] += correct_per_label[i]
                label_total[i] += len(images)

                #calc TP and FP
                true_positive = ((predicted_labels[:, i] == 1) & (labels[:, i] == 1)).sum().item()
                false_positive = ((predicted_labels[:, i] == 1) & (labels[:, i] == 0)).sum().item()
                true_negative = ((predicted_labels[:, i] == 0) & (labels[:, i] == 0)).sum().item()
                false_negative = ((predicted_labels[:, i] == 0) & (labels[:, i] == 1)).sum().item()

                TP[i] += true_positive
                FP[i] += false_positive
                TN[i] += true_negative
                FN[i] += false_negative

        for i in range(args.num_classes):

            label_accuracy.append(
                (label_correct[i] / label_total[i] if label_total[i] > 0 else 0)*100
            )

            precision.append((TP[i] / (TP[i] + FP[i]) if TP[i] + FP[i] > 0 else 0)*100)
            sensi.append((TP[i] / (TP[i] + FN[i]) if TP[i] + FN[i] > 0 else 0)*100)
            specificity.append((TN[i] / (TN[i] + FP[i]) if TN[i] + FP[i] > 0 else 0)*100)
            FNR.append((FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0)*100)
            F1.append( 2 * (precision[i] * sensi[i]) / (precision[i] + sensi[i]) if precision[i] + sensi[i] > 0 else 0)


        # Calculate macro-averages and overall validation accuracy
        validation_loss /= len(validation_loader.dataset)
        overall_accuracy = total_correct / len(validation_loader.dataset)
        validation_accuracy = (
            overall_accuracy * 100
        )

        label_accuracy.insert(0, validation_accuracy )
        precision.insert(0, (sum(precision) / args.num_classes))
        sensi.insert(0, (sum(sensi) / args.num_classes))
        specificity.insert(0, (sum(specificity) / args.num_classes))
        FNR.insert(0, (sum(FNR) / args.num_classes))
        F1.insert(0, (sum(F1) / args.num_classes))

        #Create a dictionary for validation metrics and save to DataFrame
        val_metrics_dict = {
            "Epoch": [epoch] * len(label_names),
            "Type (VAL)": label_names,
            "Mean Accuracy (VAL)": label_accuracy,
            "Precision (VAL)": precision,
            "Sensitivity (VAL)": sensi,
            "Specificity (VAL)": specificity,
            "FNR (VAL)": FNR,
            "F1 Score (VAL)": F1,
        }

        mean_accuracy = validation_accuracy
        f1_score_mean = F1[0]

        # Update the highest mean accuracy and save the model if needed
        if ((mean_accuracy > highest_mean_accuracy) or (highest_mean_accuracy == 0.0)):
            highest_mean_accuracy = mean_accuracy
            model_save_path = os.path.join(output_folder_path, 'model_highest_mean_accuracy.pth')
            torch.save(model.state_dict(), model_save_path)

        # Update the highest F1 score and save the model if needed
        if ((f1_score_mean > highest_f1_score) or (highest_f1_score == 0.0)):
            highest_f1_score = f1_score_mean
            model_save_path = os.path.join(output_folder_path, 'model_highest_f1_score.pth')
            torch.save(model.state_dict(), model_save_path)

        # Print label accuracies and overall accuracy
        print(f"Validation Overall Accuracy: {validation_accuracy:.4f}")
        print(f"Validation Loss: {validation_loss:.4f}")
        print("-------------------------------------------------")
        print("")

    return val_metrics_dict


# Function to test the model
def test_model(model, criterion, test_loader, device):
    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        total_correct = 0
        label_correct = [0] * args.num_classes
        label_total = [0] * args.num_classes
        TP = [0] * args.num_classes
        FP = [0] * args.num_classes
        TN = [0] * args.num_classes
        FN = [0] * args.num_classes
        precision, specificity, sensi, FNR, F1, label_accuracy =  [], [], [], [], [], []


        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)

            # Calculate accuracies
            predicted_labels = (torch.sigmoid(outputs) > 0.5).float()
            total_correct += (predicted_labels == labels).all(dim=1).sum().item()
            correct_per_label = (predicted_labels == labels).sum(dim=0).tolist()

            for i in range(args.num_classes):
                label_correct[i] += correct_per_label[i]
                label_total[i] += len(images)

                #calc TP and FP
                true_positive = ((predicted_labels[:, i] == 1) & (labels[:, i] == 1)).sum().item()
                false_positive = ((predicted_labels[:, i] == 1) & (labels[:, i] == 0)).sum().item()
                true_negative = ((predicted_labels[:, i] == 0) & (labels[:, i] == 0)).sum().item()
                false_negative = ((predicted_labels[:, i] == 0) & (labels[:, i] == 1)).sum().item()

                TP[i] += true_positive
                FP[i] += false_positive
                TN[i] += true_negative
                FN[i] += false_negative

        for i in range(args.num_classes):
            label_accuracy.append(
                (label_correct[i] / label_total[i] if label_total[i] > 0 else 0)*100
            )

            precision.append((TP[i] / (TP[i] + FP[i]) if TP[i] + FP[i] > 0 else 0)*100)
            sensi.append((TP[i] / (TP[i] + FN[i]) if TP[i] + FN[i] > 0 else 0)*100)
            specificity.append((TN[i] / (TN[i] + FP[i]) if TN[i] + FP[i] > 0 else 0)*100)
            FNR.append((FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0)*100)
            F1.append( 2 * (precision[i] * sensi[i]) / (precision[i] + sensi[i]) if precision[i] + sensi[i] > 0 else 0)

        overall_accuracy = total_correct / len(test_loader.dataset)
        test_accuracy = (overall_accuracy * 100)  # Multiply by 100 to get percentage
        test_loss /= len(test_loader.dataset)

        label_accuracy.insert(0, test_accuracy )
        precision.insert(0, (sum(precision) / args.num_classes))
        sensi.insert(0, (sum(sensi) / args.num_classes))
        specificity.insert(0, (sum(specificity) / args.num_classes))
        FNR.insert(0, (sum(FNR) / args.num_classes))
        F1_score = (sum(F1) / args.num_classes)
        F1.insert(0, (sum(F1) / args.num_classes))
        epoch = 1

        #Create a dictionary for test metrics and save to DataFrame
        test_metrics_dict = {
            "Epoch": [epoch] * len(label_names),
            "Type (TEST)": label_names,
            "Mean Accuracy (TEST)": label_accuracy,
            "Precision (TEST)": precision,
            "Sensitivity (TEST)": sensi,
            "Specificity (TEST)": specificity,
            "FNR (TEST)": FNR,
            "F1 Score (TEST)": F1,
        }


        # Print label accuracies and overall accuracy
        print(f"Test Overall Accuracy: {test_accuracy:.4f}")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"F1 Score: {F1_score:.4f}")
        print("-------------------------------------------------")

    return test_metrics_dict

# Function to calculate class weights based on a CSV file
def calcWeights(csv_path):
    train_df = pd.read_csv(csv_path)
    columns = train_df.keys()
    columns = list(columns)
    columns.remove("Num")
    columns.remove("ID")
    pos_count = []
    neg_count = []
    pos_weights = []
    total = 746
    for column in columns:
        pos_count.append(train_df[column].sum())
        neg_count.append(total - (train_df[column].sum()))
        pos_weights.append((total - (train_df[column].sum())) / total)

    return pos_weights


if __name__ == "__main__":

    args = argparse.Namespace(
        image_dir="/content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/Images",
        train_csv="/content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/train.csv",
        validation_csv="/content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/val.csv",
        test_csv="/content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/test.csv",
        num_classes=8,
        output_dir="/content/drive/My Drive/baselineNeckResults",
        output_file_prefix="res50",
        output_file_extension=".csv",
        )

    print("=======================================================")
    print("Arguments:")
    for arg, value in vars(args).items():
        print(f"{arg}: {value}")
    print("=======================================================")
    print("")

    #Initializations
    label_names = [
        "Overall (Macro-Average)",
        "Alignment",
        "Soft_tissue_Swelling",
        "Listhesis",
        "Fracture",
        "Dislocation",
        "Spinous",
        "Other_Pathogens",
        "normal",
    ]
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pos_weights = torch.tensor(calcWeights(args.train_csv), device=DEVICE)

    # Create the output directory if it doesn't exist
    os.makedirs(args.output_dir, exist_ok=True)

    # Initial Set of Combinations
    hyperparameter_grid = [
        {"bs": 32, "lr": 0.001, "res": 64, "epochs": 2},
        {"bs": 32, "lr": 0.001, "res": 128, "epochs": 2},
        # {"bs": 32, "lr": 0.001, "res": 64, "epochs": 15},
        # {"bs": 32, "lr": 0.001, "res": 128, "epochs": 15},
        # {"bs": 64, "lr": 0.001, "res": 64, "epochs": 15},
        # {"bs": 64, "lr": 0.001, "res": 128, "epochs": 15},
        # {"bs": 32, "lr": 0.002, "res": 64, "epochs": 15},
        # {"bs": 32, "lr": 0.002, "res": 128, "epochs": 15},
        # {"bs": 64, "lr": 0.002, "res": 64, "epochs": 15},
        # {"bs": 64, "lr": 0.002, "res": 128, "epochs": 15},
        # {"bs": 32, "lr": 0.0015, "res": 96, "epochs": 15},
    ]

    results_df = pd.DataFrame()


    for hyperparams in hyperparameter_grid:

        #Initialize hyperparameters
        batch_size = hyperparams["bs"]
        learning_rate = hyperparams["lr"]
        resolution = hyperparams["res"]
        epochs = hyperparams["epochs"]

        #initialize output directory
        output_df = pd.DataFrame()
        folder_name = f"bs{batch_size}_lr{learning_rate:.5f}_res{resolution}_ep{epochs}"
        output_folder_path = os.path.join(args.output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)
        output_file_path = os.path.join(output_folder_path, "output.csv")

        #Print Hyperparameters
        print("Batch size:", batch_size)
        print("Learning Rate:", learning_rate)
        print("Resolution:", resolution)
        print("")

        # Load data with the current hyperparameter settings
        train_loader, validation_loader, test_loader = load_data(
            args.image_dir,
            args.train_csv,
            args.validation_csv,
            args.test_csv,
            batch_size,
            resolution,
        )

        # Load the ResNet-50 model
        model = load_resnet50(args.num_classes)

        # Define loss function and optimizer
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Train the model
        train_val_df = train_model(
            model,
            criterion,
            optimizer,
            train_loader,
            validation_loader,
            epochs,
            DEVICE,
            output_folder_path,
        )
        del model

        # Test the best Mean Acc model
        best_mean_accuracy_model = load_resnet50(args.num_classes)
        best_mean_accuracy_model.load_state_dict(torch.load(os.path.join(output_folder_path, 'model_highest_mean_accuracy.pth')))
        best_mean_accuracy_model.to(DEVICE)
        mean_acc_test_metrics_dict = test_model(
            best_mean_accuracy_model, criterion, test_loader, DEVICE
        )
        del best_mean_accuracy_model

        #Test best F1-Score Model
        best_f1_score_model = load_resnet50(args.num_classes)
        best_f1_score_model.load_state_dict(torch.load(os.path.join(output_folder_path, 'model_highest_f1_score.pth')))
        best_f1_score_model.to(DEVICE)
        macro_F1_test_metrics_dict = test_model(
            best_f1_score_model, criterion, test_loader, DEVICE
        )
        del best_f1_score_model

        #accumulate results
        mean_acc_test_df = pd.DataFrame(mean_acc_test_metrics_dict)
        mean_F1_test_df = pd.DataFrame(macro_F1_test_metrics_dict)
        test_df = pd.concat([mean_acc_test_df, mean_F1_test_df ], axis=1)
        csv_df = pd.concat([train_val_df, test_df ], axis=1)

        #save individual model results
        csv_df.to_csv(output_file_path, index=False)

        #Find best Mean Accuracy and F1 Score
        overall_mean_acc = mean_acc_test_metrics_dict["Mean Accuracy (TEST)"][0]
        mac_f1 =  mean_acc_test_metrics_dict["F1 Score (TEST)"][0]
        overall_macro_F1 = macro_F1_test_metrics_dict["F1 Score (TEST)"][0]
        mac_acc = macro_F1_test_metrics_dict["Mean Accuracy (TEST)"][0]

        if mac_acc > overall_mean_acc:
            overall_mean_acc = mac_acc

        if mac_f1 > overall_macro_F1:
            overall_macro_F1 = mac_f1

        #Save Best Results to dictionary
        results_dict = {
        "hyperparameters": hyperparams,
        "test_metrics_mean_acc": overall_mean_acc,
        "test_metrics_macro_F1": overall_macro_F1
        }

        #Append dictionary to other results for same hyperparameter search
        a_df = pd.DataFrame(results_dict)
        results_df = pd.concat([results_df, a_df ], axis=0)

    #initilize hyperparameter results file
    output_file_name = "GridSearchResults"
    output_file_path = os.path.join(args.output_dir, output_file_name + ".csv")

    # Create a DataFrame from the hyperparameter search results and save to a CSV file
    results_df.to_csv(output_file_path, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arguments:
image_dir: /content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/Images
train_csv: /content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/train.csv
validation_csv: /content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/val.csv
test_csv: /content/drive/MyDrive/Medical data/Neck/Unlocalized/supervised data/test.csv
num_classes: 8
output_dir: /content/drive/My Drive/baselineNeckResults
output_file_prefix: res50
output_file_extension: .csv

Batch size: 32
Learning Rate: 0.001
Resolution: 64



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch:  0

Training Label Accuracies:
Overall Accuracy: 10.0536
Train Loss: 0.4123
F1 Score: 8.7687

Validation Overall Accuracy: 1.0753
Validation Loss: 0.5887
-------------------------------------------------

Epoch:  1

Training Label Accuracies:
Overall Accuracy: 8.5791
Train Loss: 0.3479
F1 Score: 5.6540

Validation Overall Accuracy: 3.2258
Validation Loss: 0.9137
-------------------------------------------------



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Overall Accuracy: 1.0638
Test Loss: 0.9845
F1 Score: 16.7486
-------------------------------------------------
Test Overall Accuracy: 1.0638
Test Loss: 0.9845
F1 Score: 16.7486
-------------------------------------------------
Batch size: 32
Learning Rate: 0.001
Resolution: 128



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch:  0

Training Label Accuracies:
Overall Accuracy: 3.0831
Train Loss: 0.4313
F1 Score: 5.2344

Validation Overall Accuracy: 0.0000
Validation Loss: 0.3562
-------------------------------------------------

Epoch:  1

Training Label Accuracies:
Overall Accuracy: 3.6193
Train Loss: 0.3526
F1 Score: 3.7968

Validation Overall Accuracy: 1.0753
Validation Loss: 0.3194
-------------------------------------------------



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Overall Accuracy: 0.0000
Test Loss: 0.3507
F1 Score: 0.4717
-------------------------------------------------
Test Overall Accuracy: 0.0000
Test Loss: 0.4005
F1 Score: 0.9091
-------------------------------------------------
